In [1]:
!ls

2010 Federal STEM Education Inventory Data Set.xls
Untitled.ipynb
task.txt


In [2]:
!cat task.txt

Create a public repo called ml_stem_analysis on github and share with us.
 
Data: https://data.world/education/2010-federal-stem-inventory/file/2010%20Federal%20STEM%20Education%20Inventory%20Data%20Set.xls
 
Stage 1:
1) Calculate % growth of funding between year 2008 & 2009.
2) If funding is positive, tag it as 1, if funding is negative tag it as 0. This is the target variable.
 
Stage 2:
1) Create graphs of univariate distribution of all non funding variables and share on a jupyter notebook. Just FYI - Funding FY2008, FY2009, FY2010 are the "funding variables"
2) Calculate mutual_info_score of target variable created in stage 1 & ALL non funding variables and share on a jupyter notebook.
 
Stage 3:
1) Divide data into train & test samples. (70-30 split)
2) Select features & build xgboost model. You will be judged on roc_auc_score on test sample.
3) Write testcases on all user defined functions using pytest framework. This is one of the most important steps of this interview.


In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_excel('2010 Federal STEM Education Inventory Data Set.xls')

In [5]:
df.shape

(254, 255)

In [6]:
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', 10):
    display(df.head(3))

,PROGRAM,Unnamed: 1,Unnamed: 2,Unnamed: 3,FUNDING,...,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254
Index Number,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,...,Y4) Portfolio Evaluation/Review Type,NaN,NaN,Y5) Expert Review Type,NaN
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,...,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,...,NaN,NaN,NO,NaN,NaN


In [7]:
df.columns = df.iloc[0]
df.head(3)

Index Number,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,nan,nan,Y3) Process or Implementation Evaluation Type,nan,nan,Y4) Portfolio Evaluation/Review Type,nan,nan,Y5) Expert Review Type,nan
Index Number,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,NaN,NaN,Y3) Process or Implementation Evaluation Type,NaN,NaN,Y4) Portfolio Evaluation/Review Type,NaN,NaN,Y5) Expert Review Type,NaN
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,5.24,7.19,5.7,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,14,14,14,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN


In [8]:
df = df.iloc[1:]

In [9]:
df.columns

Index([                              'Investment Name',
                                              'Agency',
                                           'Subagency',
                                'A) Brief Description',
                                 'B) Year Established',
                                  'C1) Funding FY2008',
                                  'C2) Funding FY2009',
                                  'C3) Funding FY2010',
                'D) Mission-specific or General STEM?',
       'E) Agency or Mission-Related Workforce Needs?',
       ...
                                                   nan,
                                                   nan,
       'Y3) Process or Implementation Evaluation Type',
                                                   nan,
                                                   nan,
                'Y4) Portfolio Evaluation/Review Type',
                                                   nan,
                                     

In [10]:
# Create a list for df columns as shown in excel (here, multiple spanning of columns will be shown as colname + 0/1/2....)
lis = []
k = 0
for x, y in zip(df.columns.isnull(), df.columns):
    if not x:
        z = y
        lis.append(y)
        j = 0
        k = 0
    else:
        if j == 0:
            lis.pop()
            lis.append(z + str(k))
            k += 1
            lis.append(str(z) + str(k))
        else:
            k += 1
            lis.append(str(z) + str(k))
lis[:15]

['Investment Name',
 'Agency',
 'Subagency',
 'A) Brief Description',
 'B) Year Established',
 'C1) Funding FY2008',
 'C2) Funding FY2009',
 'C3) Funding FY2010',
 'D) Mission-specific or General STEM?',
 'E) Agency or Mission-Related Workforce Needs?',
 'F1) Primary Investment Objective',
 'F2) Secondary Investment Objectives0',
 'F2) Secondary Investment Objectives1',
 'F2) Secondary Investment Objectives2',
 'F2) Secondary Investment Objectives3']

In [11]:
df.columns = lis

In [12]:
df.head(3)

,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,Y2) Summative Evaluation Type1,Y2) Summative Evaluation Type2,Y3) Process or Implementation Evaluation Type0,Y3) Process or Implementation Evaluation Type1,Y3) Process or Implementation Evaluation Type2,Y4) Portfolio Evaluation/Review Type0,Y4) Portfolio Evaluation/Review Type1,Y4) Portfolio Evaluation/Review Type2,Y5) Expert Review Type0,Y5) Expert Review Type1
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,5.24,7.19,5.7,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,14,14,14,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN
PRG_0003,CISE Pathways to Revitalized Undergraduate Com...,National Science Foundation,Directorate for Computer & Information Science...,Through the CISE Pathways to Revitalized Under...,2007,5,5,4.37,General STEM,NaN,...,NaN,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN


## Stage 1


 1) Calculate % growth of funding between year 2008 & 2009.

In [14]:
df.columns[:10]

Index(['Investment Name', 'Agency', 'Subagency', 'A) Brief Description',
       'B) Year Established', 'C1) Funding FY2008', 'C2) Funding FY2009',
       'C3) Funding FY2010', 'D) Mission-specific or General STEM?',
       'E) Agency or Mission-Related Workforce Needs?'],
      dtype='object')

In [40]:
# Some of the df['C1) Funding FY2008'] are = NaN. Assuming that the funding for those years was 0

 df[df['% Increase'].isnull()]
df[].fillna(0, inplace=True)

KeyError: '%Increase'

In [35]:
def protector(x):
    try:
        ans = (x['C3) Funding FY2010'] - x['C1) Funding FY2008']) / x['C1) Funding FY2008']
        return (ans * 100)
    except ZeroDivisionError:
        return (100)

In [36]:

df["% Increase"] = df.apply(lambda x: protector(x), axis=1)
df

,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,Y2) Summative Evaluation Type2,Y3) Process or Implementation Evaluation Type0,Y3) Process or Implementation Evaluation Type1,Y3) Process or Implementation Evaluation Type2,Y4) Portfolio Evaluation/Review Type0,Y4) Portfolio Evaluation/Review Type1,Y4) Portfolio Evaluation/Review Type2,Y5) Expert Review Type0,Y5) Expert Review Type1,% Increase
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,2002,5.24,7.19,5.7,General STEM,NaN,...,NaN,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,8.778626
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,2006,14,14,14,General STEM,NaN,...,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,0.000000
PRG_0003,CISE Pathways to Revitalized Undergraduate Com...,National Science Foundation,Directorate for Computer & Information Science...,Through the CISE Pathways to Revitalized Under...,2007,5,5,4.37,General STEM,NaN,...,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,-12.600000
PRG_0004,Opportunities for Enhancing Diversity in the G...,National Science Foundation,Directorate for Geosciences (GEO),The Opportunities for Enhancing Diversity in t...,2002,4.57,11.79,4.18,General STEM,NaN,...,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,-8.533917
PRG_0005,Geoscience Education,National Science Foundation,Directorate for Geosciences (GEO),The Geoscience Education (GeoEd) program is fo...,1999,1.63,2.74,2.02,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,23.926380
PRG_0006,Geoscience Teacher Training (GEO-Teach),National Science Foundation,Directorate for Geosciences (GEO),The Geoscience Teacher Training (GEO-Teach) pr...,2006,3,3,2.98,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,-0.666667
PRG_0007,Global Learning and Observations to Benefit th...,National Science Foundation,Directorate for Geosciences (GEO),The GLOBE Program is a hands-on international ...,1995,1.1,1.12,1.1,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,0.000000
PRG_0008,Graduate Research Fellowship Program (GRFP),National Science Foundation,Directorate for Education and Human Resources ...,The purpose of the NSF Graduate Research Fello...,1952,96.02,162.43,136.13,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,INDEPENDENT,NaN,41.772547
PRG_0009,Integrative Graduate Education and Research Tr...,National Science Foundation,Directorate for Education and Human Resources ...,IGERT is an NSF-wide program intended to meet ...,1997,64.76,77.99,69.7,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,INDEPENDENT,NaN,7.628166
PRG_0010,Graduate Teaching Fellows in K-12 Education (G...,National Science Foundation,Directorate for Education and Human Resources ...,The objectives of the GK-12 program are: 1) to...,1999,54.6,58.84,55.97,General STEM,NaN,...,NO,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,2.509158


In [39]:
df[df['% Increase'].isnull()]

,Investment Name,Agency,Subagency,A) Brief Description,B) Year Established,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,...,Y2) Summative Evaluation Type2,Y3) Process or Implementation Evaluation Type0,Y3) Process or Implementation Evaluation Type1,Y3) Process or Implementation Evaluation Type2,Y4) Portfolio Evaluation/Review Type0,Y4) Portfolio Evaluation/Review Type1,Y4) Portfolio Evaluation/Review Type2,Y5) Expert Review Type0,Y5) Expert Review Type1,% Increase
PRG_0056,Astrophysics Forum,NASA,Science Mission Directorate (SMD),NASA’s Science Mission Directorate’s (SMD) E/P...,2009,NaN,0.99304,0.995724,General STEM,NaN,...,NO,INDEPENDENT,INTERNAL,NaN,INDEPENDENT,INTERNAL,NaN,NaN,NaN,NaN
PRG_0060,Reduced Gravity Student Flight Opportunity Pro...,NASA,SMOD,The Reduced Gravity Education Flight Project p...,1995,NaN,0.355,0.355,Agency Mission Workforce,Mission-related,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PRG_0061,SOI - Summer of Innovation,NASA,Center GRC,The Summer of Innovation project seeks to adva...,2010,NaN,NaN,10,General STEM,NaN,...,NO,INDEPENDENT,INTERNAL,NaN,NaN,NaN,NO,NaN,NaN,NaN
PRG_0066,Terra,NASA,Science Mission Directorate (SMD),The Terra education portfolio for the 2010 fis...,2009,NaN,0.4529,0.3,General STEM,NaN,...,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,NaN
PRG_0079,Aura,NASA,Science Mission Directorate (SMD),The Aura mission Education and Public Outreach...,2009,NaN,0.3761,0.3715,General STEM,NaN,...,NaN,NaN,INTERNAL,NaN,INDEPENDENT,INTERNAL,NaN,NaN,INTERNAL,NaN
PRG_0080,Heliophysics E/PO Forum,NASA,Science Mission Directorate (SMD),NASA’s Science Mission Directorate’s (SMD) E/P...,2009,NaN,0.824389,0.734045,General STEM,NaN,...,NO,INDEPENDENT,INTERNAL,NaN,INDEPENDENT,INTERNAL,NaN,NaN,NaN,NaN
PRG_0083,NAS - NASA Aerospace Scholars,NASA,Center JSC,NAS uses unique Agency assets & facilities to ...,2010,NaN,NaN,0.3,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,NaN
PRG_0086,Innovation in Higher Education STEM Education,NASA,Education Office,Innovation in Higher Education STEM Education ...,2010,NaN,NaN,0.958502,Agency Mission Workforce,Mission-related,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PRG_0092,GLOBE Program,NASA,Science Mission Directorate (SMD),The GLOBE Program is a hands-on international ...,2003,NaN,4.396,3,General STEM,NaN,...,NO,INDEPENDENT,INTERNAL,NaN,NaN,NaN,NO,NaN,NaN,NaN
PRG_0098,MOD/SIM Summer Fellowships for Middle School T...,NASA,Education Office,The NASA SIMulation-Based AeroSpace Engineerin...,2010,NaN,NaN,0.387,General STEM,NaN,...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,NaN


In [ ]:
df